In [1]:
import gc
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from __future__ import division

In [2]:
def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    '''
    --------------------------------order--------------------------------
    * This file tells us which set (prior, train, test) an order belongs
    * Unique in order_id
    * order_id in train, prior, test has no intersection
    * this is the #order_number order of this user
    '''
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})

    #  order in prior, train, test has no duplicate
    #  order_ids_pri = priors.order_id.unique()
    #  order_ids_trn = train.order_id.unique()
    #  order_ids_tst = orders[orders.eval_set == 'test']['order_id'].unique()
    #  print(set(order_ids_pri).intersection(set(order_ids_trn)))
    #  print(set(order_ids_pri).intersection(set(order_ids_tst)))
    #  print(set(order_ids_trn).intersection(set(order_ids_tst)))

    '''
    --------------------------------product--------------------------------
    * Unique in product_id
    '''
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission

In [3]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

In [4]:
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    '''Create statistical columns, group by [N columns] and compute stats on [N column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       agg_dict: python dictionary

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       {real_column_name: {your_specified_new_column_name : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   'user_buy_product_times': {'prod_order_once':lambda x: sum(x==1),
                                              'prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)
    '''
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

In [5]:
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):  
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

In [6]:
path_data = ''
priors, train, orders, products, aisles, departments, sample_submission = load_data(path_data)

In [7]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [10]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [11]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


# Product part

In [12]:
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')


In [13]:
priors_orders_detail.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [14]:
priors_orders_detail.shape

(32434489, 10)

In [15]:
priors_orders_detail.loc[:,'_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1

In [16]:
priors_orders_detail.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1


In [17]:
#priors_orders_detail.to_csv("priors_orders_detail.csv", index=False)

In [18]:
#A = pd.read_csv("priors_orders_detail.csv")

In [19]:
#A.head()

In [20]:
#priors_orders_detail.tail()

In [21]:
#test_sample = priors_orders_detail.loc[:1000]

In [22]:
#test_sample.head()

In [23]:
#A = test_sample.groupby(['user_id', 'product_id']).cumcount()+1

In [24]:
#A.head(10)

In [25]:
#agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
            #'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
           # '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                   #，即该物品被用户多次购买过的次数。
#prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

In [26]:
#prd.to_csv("prd_kun.csv",index=False)

In [27]:
#prd.head()

In [28]:
prd = pd.read_csv("prd_kun.csv")

In [29]:
#sample = test_sample.copy()

In [30]:
#sample.head(20)

In [31]:
#A= sample.groupby(['product_id'])

In [32]:
#B=A.agg(agg_dict)

In [33]:
#B.head(15)

In [34]:
#B.columns = B.columns.droplevel(0)

In [35]:
#B.head()

In [36]:
#B = B.reset_index()

In [37]:
#B.head()

In [38]:
prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
prd['_prod_reorder_ratio'] = prd._prod_reorder_tot_cnts / prd._prod_tot_cnts
prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt

In [39]:
prd.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,_prod_reorder_times
0,1,276,716,1852,1136.0,0.385475,0.613391,2.586592
1,2,8,78,90,12.0,0.102564,0.133333,1.153846
2,3,36,74,277,203.0,0.486486,0.732852,3.743243
3,4,64,182,329,147.0,0.351648,0.446809,1.807692
4,5,4,6,15,9.0,0.666667,0.600000,2.500000


# User part

In [40]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [41]:
priors_orders_detail.head(7)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1
5,2398795,1,prior,2,3,7,15.0,196,1,1,2
6,2398795,1,prior,2,3,7,15.0,10258,2,0,1


In [42]:
priors_orders_detail.eval_set.value_counts()

prior    32434489
Name: eval_set, dtype: int64

In [43]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [44]:
priors.shape

(32434489, 4)

In [45]:
priors_orders_detail.shape

(32434489, 11)

In [46]:
products.shape

(49688, 4)

In [47]:
agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean'}}

In [48]:
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)

add stats features begin ......
add stats features end ......
time lapsing 0.335267782211 s 



In [49]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders
0,1,19.555555,176.0,10
1,2,15.230769,198.0,14
2,3,12.090909,133.0,12
3,4,13.750000,55.0,5
4,5,13.333333,40.0,4


In [50]:
priors_orders_detail.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1
5,2398795,1,prior,2,3,7,15.0,196,1,1,2
6,2398795,1,prior,2,3,7,15.0,10258,2,0,1
7,2398795,1,prior,2,3,7,15.0,12427,3,1,2
8,2398795,1,prior,2,3,7,15.0,13176,4,0,1
9,2398795,1,prior,2,3,7,15.0,26088,5,1,2


In [51]:
sample = priors_orders_detail.loc[:35000]

In [52]:
#sample.shape

In [53]:
#sample.head(10)

In [54]:
#sample.groupby(['user_id'])['product_id'].aggregate('count')

In [55]:
#sample.groupby(['user_id'])['product_id'].aggregate(lambda x: x.nunique())

In [56]:
#user1 = sample.ix[sample['user_id']==1]

In [57]:
#user1.head()

In [58]:
#user1.product_id.nunique()

In [59]:
#train.head()

In [60]:
#sample.groupby(['user_id'])['reordered'].aggregate(lambda x: sum(priors_orders_detail.ix[x.index,'reordered']==1))

In [61]:
#user1.head()

In [62]:
#sum(user1.reordered==1)

In [63]:
#sum(user1.reordered)

In [64]:
agg_dict_3 = {'reordered':
              {'total_reordered': 
               lambda x: sum(x==1)},
              'order_number':
              {'total_reordered_number':
               lambda x: sum(x>1)
               },
              'product_id':{'_user_total_products':'count', 
                            '_user_distinct_products': lambda x: x.nunique()}}

In [65]:
#A= sample.groupby(['user_id'])['reordered'].aggregate(lambda x: sum(x==1)).head(5)


In [66]:
#B =sample.groupby(['user_id'])['order_number'].aggregate(lambda x: sum(x>1)).head(5)

In [67]:
#A

In [68]:
#B

In [69]:
#sample.head()

In [70]:
#priors_orders_detail.head()

In [71]:
#print sample.shape, priors_orders_detail.shape

In [72]:
sas = ka_add_groupby_features_1_vs_n(sample, ['user_id'], agg_dict_3)

add stats features begin ......
add stats features end ......
time lapsing 0.272955179214 s 



In [73]:
sas.head()

,user_id,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered
0,1,54,59,18,41
1,2,182,195,102,93
2,3,78,88,33,55
3,4,14,18,17,1
4,5,26,37,23,14


In [74]:
#us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)

In [75]:
us = pd.read_csv('us.csv')

In [76]:
us.shape

(206209, 6)

In [77]:
us.head()

,user_id,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio
0,1,54,59,18,41,0.759259
1,2,182,195,102,93,0.510989
2,3,78,88,33,55,0.705128
3,4,14,18,17,1,0.071429
4,5,26,37,23,14,0.538462


In [78]:
sas.head()

,user_id,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered
0,1,54,59,18,41
1,2,182,195,102,93
2,3,78,88,33,55
3,4,14,18,17,1
4,5,26,37,23,14


In [79]:
#sas.to_csv("sample.csv",index=False)

In [80]:
us.shape

(206209, 6)

In [81]:
#us.to_csv("us.csv",index=False)

In [82]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders
0,1,19.555555,176.0,10
1,2,15.230769,198.0,14
2,3,12.090909,133.0,12
3,4,13.750000,55.0,5
4,5,13.333333,40.0,4


In [83]:
us.tail()

,user_id,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio
206204,206205,15,32,24,8,0.533333
206205,206206,281,285,150,135,0.480427
206206,206207,199,223,92,131,0.658291
206207,206208,665,677,198,479,0.720301
206208,206209,116,129,68,61,0.525862


In [84]:
users = users.merge(us, how='inner')

In [85]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio
0,1,19.555555,176.0,10,54,59,18,41,0.759259
1,2,15.230769,198.0,14,182,195,102,93,0.510989
2,3,12.090909,133.0,12,78,88,33,55,0.705128
3,4,13.750000,55.0,5,14,18,17,1,0.071429
4,5,13.333333,40.0,4,26,37,23,14,0.538462


In [86]:
users.shape

(206209, 9)

In [87]:
users['_user_average_basket'] = users._user_total_products / users._user_total_orders

In [88]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio,_user_average_basket
0,1,19.555555,176.0,10,54,59,18,41,0.759259,5.900000
1,2,15.230769,198.0,14,182,195,102,93,0.510989,13.928571
2,3,12.090909,133.0,12,78,88,33,55,0.705128,7.333333
3,4,13.750000,55.0,5,14,18,17,1,0.071429,3.600000
4,5,13.333333,40.0,4,26,37,23,14,0.538462,9.250000


In [89]:
us.head()

,user_id,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio
0,1,54,59,18,41,0.759259
1,2,182,195,102,93,0.510989
2,3,78,88,33,55,0.705128
3,4,14,18,17,1,0.071429
4,5,26,37,23,14,0.538462


In [90]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [91]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [92]:
us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]

In [93]:
us.head()

,user_id,order_id,eval_set,days_since_prior_order
10,1,1187899,train,14.0
25,2,1492625,train,30.0
38,3,2774568,test,11.0
44,4,329954,test,30.0
49,5,2196797,train,6.0


In [94]:
us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

In [95]:
users.shape

(206209, 10)

In [96]:
us.shape

(206209, 4)

In [97]:


users = users.merge(us, how='inner')

In [98]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,19.555555,176.0,10,54,59,18,41,0.759259,5.900000,1187899,train,14.0
1,2,15.230769,198.0,14,182,195,102,93,0.510989,13.928571,1492625,train,30.0
2,3,12.090909,133.0,12,78,88,33,55,0.705128,7.333333,2774568,test,11.0
3,4,13.750000,55.0,5,14,18,17,1,0.071429,3.600000,329954,test,30.0
4,5,13.333333,40.0,4,26,37,23,14,0.538462,9.250000,2196797,train,6.0


In [99]:
users.shape

(206209, 13)

In [100]:
prd.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,_prod_reorder_times
0,1,276,716,1852,1136.0,0.385475,0.613391,2.586592
1,2,8,78,90,12.0,0.102564,0.133333,1.153846
2,3,36,74,277,203.0,0.486486,0.732852,3.743243
3,4,64,182,329,147.0,0.351648,0.446809,1.807692
4,5,4,6,15,9.0,0.666667,0.600000,2.500000


In [101]:
prd.shape

(49677, 8)

In [102]:
#users.to_csv("users",index = False)

In [103]:
priors_orders_detail.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1


In [104]:
agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                              '_up_first_order_number': 'min', 
                              '_up_last_order_number':'max'}, 
              'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                      group_columns_list=['user_id', 'product_id'], 
                                                      agg_dict=agg_dict_4)

add stats features begin ......
add stats features end ......
time lapsing 14.7048490047 s 



In [105]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number
0,1,196,1.400000,10,1,10
1,1,10258,3.333333,9,2,10
2,1,10326,5.000000,1,5,5
3,1,12427,3.300000,10,1,10
4,1,13032,6.333333,3,2,10


In [106]:
data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')

In [107]:
data.shape

(13307953, 25)

In [108]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_total_orders,total_reordered_number,_user_total_products,_user_distinct_products,total_reordered,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,10,54,59,18,41,0.759259,5.9,1187899,train,14.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,10,54,59,18,41,0.759259,5.9,1187899,train,14.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,10,54,59,18,41,0.759259,5.9,1187899,train,14.0
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,10,54,59,18,41,0.759259,5.9,1187899,train,14.0
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,10,54,59,18,41,0.759259,5.9,1187899,train,14.0


In [109]:
data['_up_order_rate'] = data._up_order_count / data._user_total_orders
data['_up_order_since_last_order'] = data._user_total_orders - data._up_last_order_number
data['_up_order_rate_since_first_order'] = data._up_order_count / (data._user_total_orders - data._up_first_order_number + 1)

In [110]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_distinct_products,total_reordered,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,18,41,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.9,0,1.000000
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.1,5,0.166667
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,18,41,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.3,0,0.333333


In [111]:
#data = pd.read_csv("data.csv")

In [112]:
fruit_dict = {}
for name in aisles.aisle.tolist():
    if name.find('fruits')!=-1:
        fruit_dict[name] = aisles[aisles.aisle==name]['aisle_id'].values[0]
        
vegetable_dict = {}
for name in aisles.aisle.tolist():
    if name.find('vegetables')!=-1:
        vegetable_dict[name] = aisles[aisles.aisle==name]['aisle_id'].values[0]

In [113]:
fruit_dict

{'bulk dried fruits vegetables': 18,
 'fresh fruits': 24,
 'packaged vegetables fruits': 123}

In [114]:
vegetable_dict

{'bulk dried fruits vegetables': 18,
 'canned jarred vegetables': 81,
 'fresh vegetables': 83,
 'packaged vegetables fruits': 123}

In [115]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [116]:
product = products.copy()
product['Organic'] = [1 if name.find('Organic')!=-1 else 0 for name in product.product_name.values]
product['fruit'] = [1 if aisle_id in fruit_dict.values() else 0 for aisle_id in product.aisle_id.values]
product['vegetable'] = [1 if aisle_id in vegetable_dict.values() else 0 for aisle_id in product.aisle_id.values]

In [117]:
prd0 = product.drop('product_name', axis=1, inplace=False)
print prd0.shape
prd0.head()

(49688, 6)


,product_id,aisle_id,department_id,Organic,fruit,vegetable
0,1,61,19,0,0,0
1,2,104,13,0,0,0
2,3,94,7,0,0,0
3,4,38,1,0,0,0
4,5,5,13,0,0,0


In [118]:
print data.shape
data.head()

(13307953, 28)


,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_distinct_products,total_reordered,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,18,41,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.9,0,1.000000
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.1,5,0.166667
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,18,41,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,18,41,0.759259,5.9,1187899,train,14.0,0.3,0,0.333333


In [119]:
data=data.merge(prd0,how='left',on='product_id')

In [120]:
data.shape

(13307953, 33)

In [121]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,train,14.0,1.0,0,1.000000,77,7,0,0,0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,train,14.0,0.9,0,1.000000,117,19,0,0,0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,train,14.0,0.1,5,0.166667,24,4,1,1,0
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,train,14.0,1.0,0,1.000000,23,19,0,0,0
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,train,14.0,0.3,0,0.333333,121,14,0,0,0


In [122]:
train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [123]:
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

print data.shape
data.head()

(13307953, 34)


,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,14.0,1.0,0,1.000000,77,7,0,0,0,1.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,14.0,0.9,0,1.000000,117,19,0,0,0,1.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,14.0,0.1,5,0.166667,24,4,1,1,0,NaN
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,14.0,1.0,0,1.000000,23,19,0,0,0,NaN
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,14.0,0.3,0,0.333333,121,14,0,0,0,1.0


In [124]:
del priors_orders_detail, orders
gc.collect()

469

In [125]:
del(sample)

In [126]:
import xgboost

train = data.loc[data.eval_set == "train",:]
train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)
train.loc[:, 'reordered'] = train.reordered.fillna(0)

X_test = data.loc[data.eval_set == "test",:]

/Users/tianbaili/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tianbaili/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [127]:
train.head()

,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
0,1.400000,10,1,10,4660,8000,35791,27791.0,0.582500,0.776480,...,14.0,1.0,0,1.000000,77,7,0,0,0,1.0
1,3.333333,9,2,10,308,557,1946,1389.0,0.552962,0.713772,...,14.0,0.9,0,1.000000,117,19,0,0,0,1.0
2,5.000000,1,5,5,1003,1923,5526,3603.0,0.521581,0.652009,...,14.0,0.1,5,0.166667,24,4,1,1,0,0.0
3,3.300000,10,1,10,889,1679,6476,4797.0,0.529482,0.740735,...,14.0,1.0,0,1.000000,23,19,0,0,0,0.0
4,6.333333,3,2,10,617,1286,3751,2465.0,0.479782,0.657158,...,14.0,0.3,0,0.333333,121,14,0,0,0,1.0


In [129]:
X_test.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
18,15,196,2.200,5,15,22,4660,8000,35791,27791.0,...,7.0,0.227273,0,0.625000,77,7,0,0,0,NaN
19,15,12427,2.100,10,1,20,889,1679,6476,4797.0,...,7.0,0.454545,2,0.454545,23,19,0,0,0,NaN
20,15,1747,3.500,4,8,19,221,562,1448,886.0,...,7.0,0.181818,3,0.266667,23,19,0,0,0,NaN
21,15,10441,2.375,8,1,22,465,867,2909,2042.0,...,7.0,0.363636,0,0.363636,117,19,0,0,0,NaN
22,15,11266,1.600,10,1,19,645,1081,4081,3000.0,...,7.0,0.454545,3,0.454545,117,19,0,0,0,NaN


In [130]:
print train.columns, X_test.columns

Index([u'_up_average_cart_position', u'_up_order_count',
       u'_up_first_order_number', u'_up_last_order_number',
       u'_prod_buy_second_time_total_cnt', u'_prod_buy_first_time_total_cnt',
       u'_prod_tot_cnts', u'_prod_reorder_tot_cnts', u'_prod_reorder_prob',
       u'_prod_reorder_ratio', u'_prod_reorder_times',
       u'_user_mean_days_since_prior_order',
       u'_user_sum_days_since_prior_order', u'_user_total_orders',
       u'total_reordered_number', u'_user_total_products',
       u'_user_distinct_products', u'total_reordered', u'_user_reorder_ratio',
       u'_user_average_basket', u'time_since_last_order', u'_up_order_rate',
       u'_up_order_since_last_order', u'_up_order_rate_since_first_order',
       u'aisle_id', u'department_id', u'Organic', u'fruit', u'vegetable',
       u'reordered'],
      dtype='object') Index([u'user_id', u'product_id', u'_up_average_cart_position',
       u'_up_order_count', u'_up_first_order_number', u'_up_last_order_number',
       u'_

In [139]:
test_size = 0.2
num_boost_round=200

In [137]:
X_test.head(5)

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
18,15,196,2.200,5,15,22,4660,8000,35791,27791.0,...,7.0,0.227273,0,0.625000,77,7,0,0,0,NaN
19,15,12427,2.100,10,1,20,889,1679,6476,4797.0,...,7.0,0.454545,2,0.454545,23,19,0,0,0,NaN
20,15,1747,3.500,4,8,19,221,562,1448,886.0,...,7.0,0.181818,3,0.266667,23,19,0,0,0,NaN
21,15,10441,2.375,8,1,22,465,867,2909,2042.0,...,7.0,0.363636,0,0.363636,117,19,0,0,0,NaN
22,15,11266,1.600,10,1,19,645,1081,4081,3000.0,...,7.0,0.454545,3,0.454545,117,19,0,0,0,NaN


In [138]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,14.0,1.0,0,1.000000,77,7,0,0,0,1.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,14.0,0.9,0,1.000000,117,19,0,0,0,1.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,14.0,0.1,5,0.166667,24,4,1,1,0,NaN
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,14.0,1.0,0,1.000000,23,19,0,0,0,NaN
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,14.0,0.3,0,0.333333,121,14,0,0,0,1.0


In [140]:
# subsample 让training时间更短
X_train, X_val, y_train, y_val = train_test_split(train.drop('reordered', axis=1), train.reordered,
                                                    test_size=test_size, random_state=42)
d_train = xgboost.DMatrix(X_train, y_train)
xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}

watchlist= [(d_train, "train")]
bst = xgboost.train(params=xgb_params, dtrain=d_train, num_boost_round=num_boost_round, evals=watchlist, verbose_eval=10)

print 'training is done'

[0]	train-logloss:0.625602
[10]	train-logloss:0.335801
[20]	train-logloss:0.268785
[30]	train-logloss:0.25165
[40]	train-logloss:0.247153
[50]	train-logloss:0.245818
[60]	train-logloss:0.245217
[70]	train-logloss:0.244855
[80]	train-logloss:0.244598
[90]	train-logloss:0.244413
[100]	train-logloss:0.244236
[110]	train-logloss:0.244097
[120]	train-logloss:0.243955
[130]	train-logloss:0.243831
[140]	train-logloss:0.24373
[150]	train-logloss:0.243616
[160]	train-logloss:0.24352
[170]	train-logloss:0.243413
[180]	train-logloss:0.243316
[190]	train-logloss:0.243235
training is done


In [141]:
d_test = xgboost.DMatrix(X_test.drop(['eval_set', 'user_id', 'order_id', 'reordered', 'product_id'], axis=1))

In [142]:
result = bst.predict(d_test)

In [144]:
result

array([ 0.5757432 ,  0.33464962,  0.15817386, ...,  0.69386399,
        0.93538702,  0.63985562], dtype=float32)

In [145]:
p =0.23

In [146]:
X_test.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,aisle_id,department_id,Organic,fruit,vegetable,reordered
18,15,196,2.200,5,15,22,4660,8000,35791,27791.0,...,7.0,0.227273,0,0.625000,77,7,0,0,0,NaN
19,15,12427,2.100,10,1,20,889,1679,6476,4797.0,...,7.0,0.454545,2,0.454545,23,19,0,0,0,NaN
20,15,1747,3.500,4,8,19,221,562,1448,886.0,...,7.0,0.181818,3,0.266667,23,19,0,0,0,NaN
21,15,10441,2.375,8,1,22,465,867,2909,2042.0,...,7.0,0.363636,0,0.363636,117,19,0,0,0,NaN
22,15,11266,1.600,10,1,19,645,1081,4081,3000.0,...,7.0,0.454545,3,0.454545,117,19,0,0,0,NaN


In [147]:
X_test.loc[:,'reordered'] = (result > p).astype(int)
X_test.loc[:, 'product_id'] = X_test.product_id.astype(str)

In [148]:
submit = X_test[X_test.reordered == 1].groupby('order_id')['product_id'].agg(lambda x: ' '.join(set(x))).reset_index()
print submit.shape
submit.head()

(69991, 2)


,order_id,product_id
0,17,21463 13107
1,34,2596 47792 44663 21137 39475 16083 43504 47766
2,137,23794 38689 41787 25890 24852 5134
3,182,47672 9337 47209 13629 39275 11520 33000 32109...
4,257,27104 45013 24838 30233 21137 39475 37646 2983...


In [149]:
output = sample_submission[['order_id']].merge(submit, how='left').fillna('None')
output.columns = sample_submission.columns.tolist()
print output.shape
output.head()

(75000, 2)


,order_id,products
0,17,21463 13107
1,34,2596 47792 44663 21137 39475 16083 43504 47766
2,137,23794 38689 41787 25890 24852 5134
3,182,47672 9337 47209 13629 39275 11520 33000 32109...
4,257,27104 45013 24838 30233 21137 39475 37646 2983...


In [150]:
output.to_csv("Instacart sub1.csv", index=False)
print 'Done'

Done
